## Task 1 (6 marks)
In order to begin playing this game, we require a function that can update the game state after a move has been made.  Write a function ``add_coin(board, coin, column)``.  This function should take the following parameters:
- ``board``: A list of lists representing the current state of the board
- ``coin``: The character representing the current player's coin, i.e. 'R' or 'Y'
- ``column``: An integer representing the column in which the player is inserting their coin, with 0 representing the left-most column in the grid.  You may assume that a value greater than the maximum number of columns in the grid will never be entered.
Your function should return a list of lists representing the new state of the board after the coin has been played.

Here are some examples you can use to call your function
```python
add_coin([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]], 'R', 2)
>>> [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 'R', 0]]
add_coin([[0,0,0,0,0],[0,0,0,0,0],['R',0,0,0,0],['Y','R',0,'R','Y']], 'Y', 1)
>>> [[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 ['R', 'Y', 0, 0, 0],
 ['Y', 'R', 0, 'R', 'Y']]
```

In [2]:
def add_coin(board, coin, column) :
    # let c represents column
    c = column 
    
    #Start searching from last row, represented by 'r'
    for r in range(len(board)-1,-1,-1): 
        
        #Look for an available empty spot
        if board[r][c] == 0:
            #Replace the value of current row with the new value of coin
            board[r] = board[r][:c]+[coin]+board[r][c+1:]
            #exiting loop and update board
            return board

## Task 2 (6 marks)
To ascertain whether the game is finished, we need a function to determine whether a particular player has won.  Write a function ``is_winner(board, coin)``.  This function should take the following parameters:
- ``board``: A list of lists representing the current state of the board
- ``coin``: The character representing the current player's coin, i.e. 'R' or 'Y'
Your function should return ``True`` if the player using the ``coin`` coins has won the game in the current board position and ``False`` otherwise.

Here are some examples you can use to call your function:
```python
is_winner([[0,0,0,0,0],[0,0,0,0,0],['R',0,0,0,0],['Y','R',0,'R','Y']], 'Y')
>>> False
is_winner([['Y',0,0,0],['Y',0,0,0],['Y',0,'R','R'],['Y',0,'R','R']], 'R')
>>> True
```

In [3]:
def is_winner(board, coin) :
    #Check each row in board where 'r' represents row
    for r in range(len(board)-1):
        #Check each column in each row where 'c' represents column
        for c in range(len(board[r])-1):
            #Check if coin are all four spots
            if coin == board[r][c] and coin == board[r][c+1] \
                and coin == board[r+1][c] and coin == board[r+1][c+1]:
                    #Return true if our coin in all four spots in a square.
                    return True
    #Return false if none of our coins in all four spots in any square
    return False

## Task 3 (9 marks)
With these essential functions in place, we now wish to work towards building a competent AI to play our Connect Square game.  The concept of a <i>heuristic</i> is central to building an AI for most strategy games.  A heuristic is a function that maps a particular game state to a numeric value, indicating how desirable that state is to a particular player.  For example, a game state in which the player is about to win could be assigned a very high heuristic value, while a game state in which the player is about to lose could be assigned a very low value.

There are numerous ways in which we can define a heuristic for any particular game, but we will adopt the following approach:
- We will consider each overlapping 2 x 2 square within the grid.  For example, the points (0,0), (0, 1), (1, 0) and (1,1) will represent one square.  (1,0), (1, 1), (2, 0) and (2,1) will represent a second square and so on.
- Consider the four points making up each square:

	-  If any one of those points contains an opponent's coin, it will be impossible to win the game by filling this square with our own coins and the square will therefore be assigned a value of 0.  
	- If one of the points contains our coin and the other three are empty the square will be assigned a value of 1. 
	- If two of the points contain our coins and the other two points are empty then the square will be assigned a value of 10.
	- If three of the points contain our coins and the other one is empty then the square will be assigned a value of 100.
	- If all four of the points contain our coins then we have won the game and the square will be assigned a value of 1000.
- The heuristic value for this game state is the sum of the value of each square in the grid.
Note that there are some drawbacks to using this heuristic.  In particular, we don't consider how close our opponent is to completing a square so an AI that uses this heuristic will not try to prevent an opponent from completing his square.  You might like to consider how you could improve upon this heuristic, but for this task you should implement the heuristic as described.  Write a function ``heuristic(board, coin)`` that returns the heuristic value of the ``board`` for the player using the ``coin`` coins.

Here are some examples you can use to call your function:

```python
heuristic([[0,0,0,0,0],[0,0,0,0,0],['R',0,0,0,0],['Y','R',0,'R','Y']], 'R')
>>> 3
heuristic([['Y',0,0,0],['Y',0,0,0],['Y',0,'R','R'],['Y',0,'R','R']], 'R')
>>> 1021
```

In [4]:
def heuristic(board, coin) :
    #Initialise heuristic value to zero
    h_val = 0
    
    #Loop through each square
    for r in range(len(board)-1):
        for c in range(len(board[r])-1):
            #set a list of the points in 2 x 2 grid for each square
            square = [board[r][c], board[r][c+1], board[r+1][c], board[r+1][c+1]]
            
            #If not any opponent coins but only our coin in a square 
            if coin in square: 
                #Add value to the total value depends on how many our coins in a square
                h_val += 10**(square.count(coin)-1)
    return h_val

## Task 4 (9 marks)
We now wish to create an AI player capable of playing (and hopefully winning) our Connect Square game.  Write a function ``ai_move(board, coin)`` to play the Connect Square game.  Your function should generate a list of all possible moves from the current ``board``.  Your function should then generate the board states that arise from making each of these moves and select the move that results in the state with the highest heuristic value.  If two states have the same heuristic value, then select the one that results from the using the numerically lowest column (e.g. column 1 is preferred over column 2 if the heuristic values are identical).  Finally, your function should return the new board state after applying the best move.

Here are some examples you can use to call your function:

```python
ai_move([[0,0,0,0,0],[0,0,0,0,0],['R',0,0,0,0],['Y','R',0,'R','Y']], 'Y')
>>> [[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 ['R', 0, 0, 'Y', 0],
 ['Y', 'R', 0, 'R', 'Y']]
ai_move([[0,0,0,0,0],['Y',0,0,0,0],['R',0,0,0,'Y'],['Y','R',0,'R','Y']], 'Y')
>>> [[0, 0, 0, 0, 0],
 ['Y', 0, 0, 0, 0],
 ['R', 0, 0, 'Y', 'Y'],
 ['Y', 'R', 0, 'R', 'Y']]
```

In [9]:
def ai_move(board, coin) :
    
    #Generate a list of all possible moves from the current board.
    all_lst = [] #Initialise a list for all possible moves
    ini_board = board #Store the original input board as an initial board
    h_val_lst = [] #Initialise a list for all heuristic values
    
    #Loop through each column
    for column in range(len(board[0])):
        
        #Reset the board as in original input value
        board = list(ini_board) 
        #Add all possible moves into one list
        all_lst.append(add_coin(board,coin,column))
        
        #Heuristic value comparison list
        h_val = heuristic(board, coin)
        #Add all heuristic values into a list
        h_val_lst.append(h_val)
    
    #Use the column that has first highest heuristic value
    column = h_val_lst.index(max(h_val_lst))
    #Reset the board as in original input value
    board = list(ini_board) 
    #Ready to add coin
    add_coin(board,coin,column)
    return board

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 'R', 0],
 ['Y', 'R', 'R', 'R', 'Y']]

## Code to run your game
The following functions will allow you to play a game against your AI.  This may be useful when debugging.

In [5]:
def moves_exist(board) :
    """
    A move can still be made if any blank space exists on the top row
    """
    if 0 in board[0] :
        return True
    return False

def nice_print(board) :
    """
    Formats the board for nicer display
    """
    for line in board :
        print(*line)
    
def play_game(rows, cols) :
    """
    Plays a game with a human player against your AI
    """
    # Instantiate an empty board
    board = [([0]*cols) for i in range(rows)]

    # Continue playing as long as a legal move can still be made
    while(moves_exist(board)) :

        # AI plays first with the red tokens
        board = ai_move(board, 'R')
        nice_print(board)

        # Check if the AI Player has won the game
        if (is_winner(board, 'R')) :
            print('AI Wins!')
            break

        # Player moves next with the yellow tokens
        player_move = input('Enter your move: ')
        board = add_coin(board, 'Y', int(player_move))
        if (is_winner(board, 'Y')) :
            print('You win!')
            break

In [ ]:
# Call play_game to play against your AI.  Useful for testing your code.
play_game(5,7)

0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 R 0 0 0 0 0


Enter your move:  5


0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 R 0 0 0 0 0
0 R 0 0 0 Y 0


Enter your move:  2


0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 R R 0 0 0 0
0 R Y 0 0 Y 0


...

0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 Y R 0 0 0 0
0 R R 0 0 0 0
0 R Y 0 0 Y 0


Enter your move:  4


0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 Y R 0 0 0 0
0 R R 0 0 0 0
R R Y 0 Y Y 0


Enter your move:  0


0 0 0 0 0 0 0
0 0 R 0 0 0 0
0 Y R 0 0 0 0
Y R R 0 0 0 0
R R Y 0 Y Y 0
